In [1]:
import MySQLdb
import pandas as pd


class DB(object):
    def __init__(self, test=False):
        # Open database connection
#         self.db = MySQLdb.connect(
#                 "localhost",
#                 "root",
#                 "root",
#                 "dbmaster")
        self.db = MySQLdb.connect(
                "127.0.0.1",
                "tcuser",
                "oochahwielai9mahDah3",
                "v2",
                port=3307)
#         self.db = MySQLdb.connect(
#                 "127.0.0.1",
#                 "root",
#                 "!qazmlp)5",
#                 "v3_1",
#                 port=3307)
        

    def to_table(self, sql):
        cursor = self.db.cursor()

        try:
           # Execute the SQL command
           cursor.execute(sql)
           df = pd.read_sql(sql,  con=self.db)
        except Exception as e:
           print "Error: unable to fecth data"
           print (e)

        # disconnect from server

        return df

    def close(self):
        self.db.close()
    


In [2]:
import datetime
dbb = DB()
inventory_df = dbb.to_table("""select * from customer_address_entity_text where attribute_id = 25""")


In [108]:
import numpy as np
import pandas as pd

loaded_csv = pd.DataFrame.from_csv("/Users/varunpasad/designated_store.csv")
loaded_csv.entity_id = loaded_csv.entity_id.astype(int)
df = loaded_csv.set_index("entity_id")


# remove all filled ones
df = df[df.store.isnull()]
print(len(df))
split_df = pd.DataFrame(df.value)
# split_df = df.value.str.split("|", expand=True)

# 0, 1
# garea_string = "Tamil Nadu, India"
# split_df["garea"] = np.where(split_df[1].str.contains(garea_string), split_df[1], None)
# split_df["garea"] = np.where(split_df[1].str.contains(garea_string), split_df[1], split_df.garea)
# remove proper google address(28352)
# split_df = split_df[-split_df.garea.isnull()]

# remove the 8 rows that have pakkam
# split_df = split_df[split_df[1].str.strip() == "Thoraipakkam"]
# split_df = split_df[split_df[0].str.strip() == "Thoraipakkam"]
# split_df = split_df[split_df[1].str.strip() == "OMR"]

# need to clean
# split_df[split_df[1].isnull()]
df

2896


,value_id,entity_type_id,attribute_id,value,store
entity_id,,,,,
1,1,2,25,Street,NaN
67,67,2,25,"door 3m,block no.3,floor no.3,radian iv terrac...",NaN
206,206,2,25,"Flat.no: FOL(4th floor), Block 8,|Jains pebble...",NaN
394,388,2,25,"F401, TVH park villa,thorraipakkam.Landmark-ne...",NaN
696,686,2,25,2b aishwarya apartment devraj nagar sholangin...,NaN
785,775,2,25,"Creek Street, Sowdashwari Nagar, Chennai, Tami...",NaN
813,803,2,25,"Kalaimagal Nagar, Chennai, Tamil Nadu, India",NaN
842,831,2,25,"5FE Jains Pebble Brook, Madha Kovil St, Viveka...",NaN
875,864,2,25,block a101 pelken nest apartment creek street ...,NaN


In [109]:

def run_google_area():
    # remove google area & strip chennai
    areas = split_df.fillna("")
#     if full_match:
#         areas = split_df.garea.str.replace("Tamil Nadu, India", "")
#         areas = areas.str.replace("Chennai,", "")
#         areas = areas.str.strip().str.split(",", expand=True)

    # load the area to store mapping
    filled_df = pd.read_csv("/Users/varunpasad/Desktop/filledata.csv")
    filled_df = filled_df.dropna()
    areas_map = filled_df.set_index('area').to_records()
    total = 0
    all_id = []

    for area_name, store_name in areas_map:
        entity_ids = []
        for col_name in areas.columns:
            # safe search !!1
            # + ","
            area_df = areas[areas[col_name].str.strip().str.contains(area_name )]
            entity_ids.extend(area_df.index.tolist())
            all_id.extend(area_df.index.tolist())
        
        total += len(entity_ids)

        if entity_ids:
            loaded_csv.loc[loaded_csv.entity_id.isin(entity_ids), 'store'] = store_name
#             print(loaded_csv[loaded_csv.entity_id.isin(entity_ids)])
    print(total)

    loaded_csv.to_csv("/Users/varunpasad/designated_store.csv")
#     print (loaded_csv[loaded_csv.entity_id.isin(all_id)])

    print(len(entity_ids))

def handle_tpak():
    filled_df = pd.read_csv("/Users/varunpasad/Desktop/filledata.csv")
    areas_map = filled_df.set_index('area').to_records()
    entity_ids = []
    for area_name, store_name in areas_map:
        entity_ids.extend(split_df[split_df[0].str.strip() == area_name].index.tolist())
    
        loaded_csv.loc[loaded_csv.entity_id.isin(entity_ids), 'store'] = store_name
    loaded_csv.to_csv("/Users/varunpasad/designated_store.csv")
    
    

    

run_google_area()
import nltk
# print(df.value.tolist())
tokens = nltk.word_tokenize(" ".join(df.value.astype(str).tolist()))
common = nltk.FreqDist(tokens)
common.most_common()

/Users/varunpasad/django/lib/python2.7/site-packages/ipykernel_launcher.py:22: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


0
0


[(',', 6812),
 ('Tamil', 989),
 ('Nadu', 988),
 ('Chennai', 868),
 ('Nagar', 468),
 ('street', 431),
 ('nagar', 389),
 (':', 373),
 ('road', 360),
 ('to', 343),
 ('Street', 273),
 ('India', 260),
 ('.', 241),
 ('Road', 207),
 ('no', 178),
 ('LM', 171),
 ('2nd', 164),
 ('main', 161),
 ('school', 156),
 ('?', 154),
 ('near', 153),
 ('apartment', 151),
 ('1st', 140),
 ('floor', 135),
 ('block', 113),
 (';', 110),
 ('Main', 103),
 ('cross', 99),
 ('lm', 96),
 ('colony', 95),
 ('opp', 90),
 ('Colony', 87),
 ('No', 83),
 ('flat', 80),
 ('3rd', 79),
 ('garden', 78),
 ('Flat', 77),
 ('India|Near', 75),
 ('STREET', 75),
 ('park', 69),
 ('ROAD', 68),
 ('NAGAR', 67),
 ('India|near', 62),
 ('hospital', 62),
 ('Door', 60),
 ('of', 59),
 ('4th', 58),
 ('2', 57),
 ('koil', 56),
 ('Cross', 56),
 ('hotel', 56),
 ('avenue', 53),
 ('bank', 52),
 ('School', 52),
 ('chennai', 52),
 ('office', 51),
 ('A', 50),
 ('Avenue', 50),
 ('city', 48),
 ('Near', 48),
 ('1', 47),
 ('Block', 46),
 ('college', 46),
 ('ap